# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load dataframe and display
output_data_file = pd.read_csv('/Users/rollycostillas/Desktop/upenn/homework/myrepository/python-api-challenge/cities.csv')
output_data_file

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bredasdorp,5,ZA,1651759786,41,-34.5322,20.0403,76.06,11.34
1,1,Carnarvon,40,AU,1651759788,56,-24.8667,113.6333,71.67,1.14
2,2,Qaanaaq,81,GL,1651759847,80,77.4840,-69.3632,17.65,8.66
3,3,Kapaa,40,US,1651759785,82,22.0752,-159.3190,73.72,16.11
4,4,Tasiilaq,0,GL,1651759872,58,65.6145,-37.6368,28.56,7.92
...,...,...,...,...,...,...,...,...,...,...
537,537,Am Timan,56,TD,1651759937,21,11.0297,20.2827,103.42,6.26
538,538,Komádi,91,HU,1651760388,44,47.0000,21.5000,73.96,1.36
539,539,Uruguaiana,20,BR,1651760245,82,-29.7547,-57.0883,64.33,9.22
540,540,Srednekolymsk,98,RU,1651759956,86,67.4500,153.6833,32.16,9.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations
locations = output_data_file[["Lat", "Lng"]]
humidity = output_data_file["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# create ideal new dataframe containing only cities with max temp between 70F & 80F, 
# cloudiness is 0%, and Wind Speed is less than 10mph
vacation_data = output_data_file.loc[(output_data_file["Max Temp"]>70) & 
                                 (output_data_file["Max Temp"]<80) & 
                                 (output_data_file["Cloudiness"]==0) &
                                 (output_data_file["Wind Speed"]<10),:]

# display dataframe
vacation_data

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
32,32,East London,0,ZA,1651759800,58,-33.0153,27.9116,76.44,9.93
36,36,Coxim,0,BR,1651760222,46,-18.5067,-54.7600,72.23,6.55
120,120,Shymkent,0,KZ,1651760247,33,42.3000,69.6000,75.79,4.47
131,131,Cabo San Lucas,0,MX,1651759961,79,22.8909,-109.9124,72.61,3.00
152,152,Viljoenskroon,0,ZA,1651760255,22,-27.2084,26.9485,72.84,4.09
182,182,Chenghai,0,CN,1651760268,64,23.4632,116.7680,73.31,4.47
266,266,Kaoma,0,ZM,1651760293,41,-14.7833,24.8000,78.08,6.35
284,284,Martinsville,0,US,1651760002,64,36.6915,-79.8725,70.18,8.05
310,310,Mariental,0,NaN,1651760313,19,-24.6333,17.9667,79.66,6.82
522,522,Roebourne,0,AU,1651760384,39,-20.7833,117.1333,79.30,9.86


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = vacation_data.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# drop any rows where hotel is still null (not found)
hotel_df = hotel_df.dropna(how="any")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "Aap_uEA6gUnyeGJMSWKBFoRv-hKS19qkzwxVd6-99bqaa6mBUec6ovIGkqNPjYks_NaeA_ichBGMeBYH0TYXKNbaWUCLa_PJIsDD0_5fJoPzmZXcOiE3E_duK2wo5vS6yi-6kK8uzL8o-mSyQUqDvxrHRTJWCOJ1W4cMBsaw8UFxRSU0jTHCNY0waqZ1l784H58qEKZtjJxxn6UhhVgJ16GXBqSDYDCNnVWWowHZ0-pz4gelaCa9at80HoiVxS1a4iPVi4W2RU26yRr4_YxWaj1SGE-qip_8wdgQ2UPIC4x1PA2da2jfipb2h1rakIUmIHNC8WGg1GBdRMWxYz1Z-xmibo7FMri2oanog-UGRj14vUY9NXTgbJUL_LBY_iZ26y34SbjXkei27z3jWDpKOcnNBEbq4ILMj1vhkyAON0BeRVf44yOSWR5S6cfHiOsGxOHN",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -32.9834877,
                    "lng": 27.9510882
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.98171257010728,
                        "lng": 27.95266547989272
                    },
                    "southwest": {
                        "lat": -32.984412

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDWRJ4HJZCj0HK6gRCjvpwhvdXBGwkk1Hsri607xwshGf8l7iPEDsJNm84o4vypHlQTUWbQc0JQ3hxnvojvvu-C_hd0WhvpLPs6AAvM06iGUUK1-EoU7KVC4au56fOvsmPSE3tMIz-ENjxLqH41ox1I0qGhvZzSqbiG2rrdZ4brQNq1G-2ycb3h_qBPD0xh4Zjf52hhGXCFct0BCExi6t0RGPMXclkNC_q6LjqhqJShT4r2rjaYOhEN1CyJSMTucO-su8je14AVde0kB6fvoZFglkryfOFQiFUggX51raamdhYgWHPgiItoLppxu--OUgZ-uxIxW0MWUEXXrXOklYbxafeM1FJ5bqWpXMS19U9yhZFOXA_KCmfN_TRVVLX8B-SUpWyEahb3xxWW05PS0ZPN73L1PjhH5QNcX-Quptj-6bE5-tHDq_dYdM6k9Y33",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -18.5102803,
                    "lng": -54.7586278
                },
                "viewport": {
                    "northeast": {
                        "lat": -18.50891037010728,
                        "lng": -54.75726017010727
                    },
                    "southwest": {
                        "lat": -18.5116

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAc-Bj_caRzLv6pNW0pfT0tQ7TcPp3-yMStdD_g4Fa0npLwIVluu-KEdP-unza_EDlGljqfiyY_7qjVQ9xe4r6-Te_jUJNoQsFnU_3gm2NHtjZJ_SBAeSOCU8UoopjtJRZpx3cUNRiLjvngt9GC_ZbqOGdCW1Qe5p1CxOmow7zcqzfEwOy05pkdhnb322k3xKET1YAjhDQevgoYmhRO2hzNFLM5pqMiwn72_EqMm1-LPFp8-i71E55x-cx7fymIayM0iQNocLxJKV8ESfvkjxWlZ-nh6GsCrNb0_5w7jlacZyEDUcu7vnICK4jN--PfMTWBRj-1NvM8pDiYnd3Gk3bYdzhrkLxs29Bs0F6Kdxn7GehJO9EBnszcArVXq-R4LQ40vin4T6dglvZm9aIISUnYpZ0iVNwJ6xM-HgLOaS65QtSSxjXwT8sReeqDa9la",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 42.3252421,
                    "lng": 69.59618739999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 42.32631917989271,
                        "lng": 69.59766557989272
                    },
                    "southwest": {
                        "lat": 42.32

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDvKAXMs-2qgYmCI7157AX8CXKSRzN_cZgUJ3mg7hhxYchNlcLv_14IB9YntQlk_h-QnfNA-NvzX7NvmG4DIvHXFmUrQ-1bVFtWeoDOwk2yqIpWsbq91f2wgQKphPxGGsv3MxJgZZAFlihzux3En4OWMS6wWXuGjoN8Z9Tk_tZ3SCicPem_lNQd6IPvxPfiwVOYjCF7u4FBRx7hDSsxfloRf1HDSgSWy08qupvHhPZfZ59BJk_YQRF7FAwsUANa-FXPpVG1deUMnA2rL2L1EhRO070x-vCgFsWjlCZ-coqSpdxZ6A8UVOUS_UNe4zUwdhbLhTVEeSNtVRGTw9UBxnQiGuvQgoz2WyydNtzsOx4EgpjjMu5Kgy7so4JRPMVVMo1pgT3FO2YfxifFU-Un4K_cPTgEz2glTYCzTVjwFHsmb5N2qviHs8TRIazVoGxT",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.8985516,
                    "lng": -109.869561
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.89953712989272,
                        "lng": -109.86714945
                    },
                    "southwest": {
                        "lat": 22.896837470107

{
    "html_attributions": [],
    "next_page_token": "Aap_uEA5B16HciH9jY06O1RrU7VitZr_QecTCc9m4ThCi4uXV3EzGZbUyjvyF9fwf-YpX3-j6EWBPnSmXAuw1EoFXgfUEaY5znV-zIBZKVYMLfOMxhKpOV8auWpb5vN-YQrpyNp5QEg7JC9dbo0xWogof-U6U9kxetRbkyDePRCKuZG3bObLxXHuedLkWpcN1-4oCKQnc48zFMtgPkf-g6S5-C6ydtpKPMafc0GOY_EzRoXncs7ppBCl7F5eRm9A3Ayyjz4VQy0qqFWgPWl2ozhFXXcR9grqGGZeHYOapgv1LGbh2PHlG9Mpu_AV8xrXIbbUlyEqxE1TJfA4MbQC74nP8Kqm0zRtKuRrbBeIMrU8MpcJTMeTODgjQaN_pRBEHB_HSjtV3diBfhZv6Qy4-_X4xJyHF7Qo5VuoeNhpn4frCjMSE7IJ6AL19LO-MOpSMtEb",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -26.8610986,
                    "lng": 26.6699729
                },
                "viewport": {
                    "northeast": {
                        "lat": -26.85978172010728,
                        "lng": 26.67157957989273
                    },
                    "southwest": {
                        "lat": -26.862481

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDBLEK441TH44ZAGWF8KaKG-G5bkgmIeY6-zJw4UygI8I1m_SyGfC7qvGRduzlUK2ccMaZS_LHiou0-IQn9ik2-jzgOffCO0eCCn-DyzpBju-_OY-jWefQTd0raznyWERWFpRrknsikRxigJFhJ3Nn0exDXlSA6Oc83ha34f7Y1brxnOSXkUgjaRMOc8G3BwDqWBFp9a12xDuiqshcRZMWR1AhJmOUHfKsX3MeDkehN_5Qi2GEr0m8b7m0oI87fm-Ji9JUIf0sc2KhCuPfM0cZ8_lq6PcqUleHT_cAiQf6aFpsCRyOgCiI6HLe-oidJ3VTxhBUw-VJIPItRftBwhKYTdgdIzh_2wqqylkgd06iDoozI7ivtfn53hFLQR7c9Yz7dWaTo7XlpHYj9fLBfEtuiNV3Mxp0zm9ezvsmE3VEm-X1kwncsI4_DdpH6RmvG",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 23.365841,
                    "lng": 116.704035
                },
                "viewport": {
                    "northeast": {
                        "lat": 23.36725037989272,
                        "lng": 116.7051766798927
                    },
                    "southwest": {
                        "lat": 23.3645507201

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -14.8214581,
                    "lng": 24.8035294
                },
                "viewport": {
                    "northeast": {
                        "lat": -14.82004837010728,
                        "lng": 24.80543017989272
                    },
                    "southwest": {
                        "lat": -14.82274802989272,
                        "lng": 24.80273052010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Mumu Chalets Garden Lodge",
            "photos": [
                {
                    "heig

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -24.6306018,
                    "lng": 17.7699413
                },
                "viewport": {
                    "northeast": {
                        "lat": -24.62866707010728,
                        "lng": 17.77153917989272
                    },
                    "southwest": {
                        "lat": -24.63136672989272,
                        "lng": 17.76883952010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Africa Safari Lodge",
            "opening_hours": {
                "open_now": true
         

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAD0OLaNr2SBAL1yekSuyoWFpWgTcknRwmipkZcviq70CSo8_oWrpMkvdMDFWP1QXoqRiuINYiN2DLlnPYawoVO6auwqPpKKIJNr9SWg9_IoakyYoNn7vMki2gNJfKC28ZUKD6tYzxFXHPlFyQFFzB4ToKyLCso_3Wu4ADaVSoLvWVP_d6sTxGUhx3fjmsyXaKZoRTOntlDDeXmA3_0sr-7sJdDMdFBpy2s24VyG0AbpLQLFGFqXbZVR1SF0WMglBz8J1KJYZebcpY771CbdECZ5Oetd7RgMLXtnNb9nP-5cCnd2F-F_TTzlpjmj8j76ujUhACVgtQ_gmFvQutnOYc9Sh_37zJO0PQYN4Fk2gAUOeyHekduoExsxhHB-B4_BBhPCwa7IgLLOIbuDNXLN-AODXyf6i2V6_s7EOTXO5m9tfgiYZDtEzhFWAvSW13w",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.7376505,
                    "lng": 116.8417759
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.73621877010728,
                        "lng": 116.8431241798928
                    },
                    "southwest": {
                        "lat": -20.73891

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,32,East London,0,ZA,1651759800,58,-33.0153,27.9116,76.44,9.93,Blue Lagoon Hotel
1,36,Coxim,0,BR,1651760222,46,-18.5067,-54.7600,72.23,6.55,Hotel Rio
2,120,Shymkent,0,KZ,1651760247,33,42.3000,69.6000,75.79,4.47,Rixos Khadisha Shymkent
3,131,Cabo San Lucas,0,MX,1651759961,79,22.8909,-109.9124,72.61,3.00,"The Cape, A Thompson Hotel"
4,152,Viljoenskroon,0,ZA,1651760255,22,-27.2084,26.9485,72.84,4.09,Protea Hotel by Marriott Klerksdorp
5,182,Chenghai,0,CN,1651760268,64,23.4632,116.7680,73.31,4.47,Shantou International Hotel
6,266,Kaoma,0,ZM,1651760293,41,-14.7833,24.8000,78.08,6.35,Mumu Chalets Garden Lodge
7,284,Martinsville,0,US,1651760002,64,36.6915,-79.8725,70.18,8.05,"Primland Resort, Auberge Resorts Collection"
9,522,Roebourne,0,AU,1651760384,39,-20.7833,117.1333,79.30,9.86,Karratha International Hotel


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))